In [ ]:
!nvidia-smi

Sat Sep  7 09:23:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9f942173d6d11978e24b87c67c53d52fc34db0db8b2bfec0a3033dfdea5ea742
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
import os
os.chdir('/content/drive/MyDrive/amazon_hackathon_2023')

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("archive/dataset/train.csv")
test = pd.read_csv("archive/dataset/test.csv")

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249698 entries, 0 to 2249697
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PRODUCT_ID       int64  
 1   TITLE            object 
 2   BULLET_POINTS    object 
 3   DESCRIPTION      object 
 4   PRODUCT_TYPE_ID  int64  
 5   PRODUCT_LENGTH   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 103.0+ MB


In [ ]:
train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [ ]:
test.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


# Data Analysis and Visualization

- tokensize <br>
- remove stop words, puncuation, special characters from title, bullet points, desciption <br>
-importance <br>
  title>bullet points> desciption <br>
-vectorize


### dataset imbalance
- when both desciption and bullet point is empty
- when title isnt there
- when either of bullet points and desciption is empty

In [ ]:
#remove duplicates
print(train.duplicated().sum())
#################################################NO DUPLICATE##################################################

0


In [ ]:
print(train['TITLE'].isna().sum())
print(train['BULLET_POINTS'].isna().sum())
print(train['DESCRIPTION'].isna().sum())
#length,height,width, weight to predict

13
837366
1157382


In [ ]:
#remove rows where all values are empty

# Create a boolean mask where all three columns are null
mask = train[['TITLE', 'DESCRIPTION', 'BULLET_POINTS']].isnull().all(axis=1)

# Count the number of such rows
count_null_rows = mask.sum()
print(f"Number of rows where all three columns are null: {count_null_rows}")

# Remove the rows where all three columns are null
train = train[~mask]


Number of rows where all three columns are null: 11


In [ ]:
#subsitute null values to empty string
train['BULLET_POINTS'].fillna('', inplace = True)
train['DESCRIPTION'].fillna('', inplace = True)

In [ ]:
import numpy as np

# Creating the new column 'x'
train['x'] = np.where(
    train['TITLE'].isnull(),
    train['DESCRIPTION'] + ' ' + train['BULLET_POINTS'],
    train['TITLE'] + ' ' + train['BULLET_POINTS'] + ' ' + train['DESCRIPTION']
)

In [ ]:
#!pip install pandas nltk spacy

In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load the spacy model
nlp = spacy.load('en_core_web_sm')

# Define stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def remove_emojis(text):
    # Regular expression for removing emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"  # other symbols
        u"\U000024C2-\U0001F251"  # enclosed characters
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):
    # Remove emojis
    text = remove_emojis(text)

    # Remove punctuation and special characters
    text = re.sub(f"[{string.punctuation}]", " ", text)
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatize the tokens
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc]

    return " ".join(tokens)

In [ ]:
train['x'] = train['x'].apply(preprocess_text)

In [ ]:
train.head()

,x,PRODUCT_LENGTH
195,die Dada Internationale Der Dadaismus Berlin U...,598.424
196,Natural Health Care Smoke Free Herbal Capsules...,196.850
197,CityPostersPlus Kaptanganj Mouse pad Mouse pad...,11.811
198,Wild Bobby Straight Outta Cleveland CLE Fan Fa...,2400.000
199,Design Art PT12719 16 32 Herd Antelope Grassla...,100.000


In [ ]:
test['x'] = test['TITLE'] + ' ' + test['BULLET_POINTS'] + ' ' + test['DESCRIPTION']
test['x'] = test['x'].apply(preprocess_text)

In [ ]:
test.head()

### tfidf


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(train['x'])
tfidf_train_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())


In [ ]:
tfidf_matrix_2 = vectorizer.fit_transform(test['x'])
tfidf_test_df = pd.DataFrame(tfidf_matrix_2.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
tfidf_train_df

,00,000,0038,01,01396096,03,077,09,095001,10,...,zircon,zollyss,zone,zu,zur,zuru,zz,â15,ât,écriture
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_test_df

### Word2vec

# Model

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVR(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(tfidf_train_df, train['PRODUCT_LENGTH'])

#Predict the response for test dataset
y_pred = clf.predict(tfidf_test_df)

In [ ]:
y_pred.head()

In [ ]:
#from sklearn import metrics
#print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred)) # Use an appropriate metric for regression problems like MSE

Mean Squared Error: 2224048.4170709974
